In [13]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain.tools import tool

In [15]:
@tool
def add_numbers(a: int, b: int) -> int:     
    """Add two numbers"""
    return a + b                            

@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [16]:
tools = [add_numbers, multiply_numbers]

In [17]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [21]:
answer = llm_with_tools.invoke('1 + 2 = ?')

In [22]:
answer

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Uu1LbADCf30ez4gL0hBy7yEy', 'function': {'arguments': '{"a":1,"b":2}', 'name': 'add_numbers'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 73, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-37b950a6-4975-4a34-8369-8f551c72ced9-0', tool_calls=[{'name': 'add_numbers', 'args': {'a': 1, 'b': 2}, 'id': 'call_Uu1LbADCf30ez4gL0hBy7yEy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 73, 'output_tokens': 18, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [23]:
answer.tool_calls

[{'name': 'add_numbers',
  'args': {'a': 1, 'b': 2},
  'id': 'call_Uu1LbADCf30ez4gL0hBy7yEy',
  'type': 'tool_call'}]

In [24]:
# llm_with_tools으로 도구를 실행해서 나온 결과를 도구 목록이 정의 된 JsonOutputToolsParser(tools=tools)에 연결
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

In [25]:
tool_call_results = chain.invoke('1 + 2 = ?')

In [26]:
tool_call_results

[{'args': {'a': 1, 'b': 2}, 'type': 'add_numbers'}]

In [28]:
single_result = tool_call_results[0]

In [30]:
print(single_result)

{'args': {'a': 1, 'b': 2}, 'type': 'add_numbers'}


In [31]:
print(single_result["type"])    # 도구 이름

add_numbers


In [32]:
print(single_result["args"])    # 도구 인자

{'a': 1, 'b': 2}


In [37]:
for tool_call_result in tool_call_results:
    tool_name = tool_call_result["type"]  # 도구의 이름(함수명)
    tool_args = tool_call_result["args"]  # 도구에 전달되는 인자

    # print(tool_name)
    # print(tool_args)

    # 도구 이름과 일치하는 도구를 찾아 실행합니다.
    matching_tool = None

    for tool in tools:
        if tool.name == tool_name:
            matching_tool = tool
            break

    if matching_tool:       # 일치하는 도구를 찾은경우.   
        result = matching_tool.invoke(tool_args)    # 해당 도구를 실행합니다.       
        print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")    # 실행 결과를 출력합니다.
    else:                   # 일치하는 도구를 찾지 못 찾은경우.        
        print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")

[실행도구] add_numbers [Argument] {'a': 1, 'b': 2}
[실행결과] 3
